In [132]:
from gensim.models import KeyedVectors
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, LSTM, TimeDistributed, Activation
import keras_metrics

import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [136]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [17]:
cn_file = './data/numberbatch-en-17.06.txt'

cn_vectors = KeyedVectors.load_word2vec_format(cn_file, binary=False)

In [6]:
def split_tags(string):
    return [tuple(i.split("/")) for i in string.split()]

def readTrainData(filename):
    data = []
    for line in open(filename):
        line = line.strip()
        #read in training or dev data with labels
        if len(line.split('\t')) == 7:
            (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = \
            line.split('\t')
        else:
            continue
        # ignoring the training data that has middle label 
        nYes = eval(judge)[0]            
        if nYes >= 3:
            amt_label = True
            data.append((split_tags(origsenttag), split_tags(candsenttag), amt_label))
        elif nYes <= 1:
            amt_label = False
            data.append((split_tags(origsenttag), split_tags(candsenttag), amt_label))
    return data

def readTestData(filename):
    data = []
    for line in open(filename):
        line = line.strip()
        #read in training or dev data with labels
        if len(line.split('\t')) == 7:
            (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = \
            line.split('\t')
        else:
            continue
        # ignoring the training data that has middle label 
        nYes = int(judge[0])
        if nYes >= 4:
            expert_label = True
        elif nYes <= 2:
            expert_label = False
        else:
            expert_label = None
        data.append((split_tags(origsenttag), split_tags(candsenttag), expert_label))
    return data

In [16]:
dev_data = readTrainData("./data/data/dev.data")
train_data = readTrainData("./data/data/train.data")
test_data = readTestData("./data/data/test.data")

len(dev_data)  #4142
len(test_data)  #972
len(train_data)  #11530

11530

In [218]:
def getTweetEmbedding(tweet):
    word_vectors = []
    exceptions = 0
    for word in tweet:
        try:
            word_vector = cn_vectors.get_vector(word[0].lower())
            word_vectors.append(word_vector)
        except:
            pass
        
    return np.mean(np.array(word_vectors), axis=0), exceptions


def getLabel(label):
    if label == True:
        return 1
    else:
        return 0
    

def getLabelsFeatures(data):
    labels, features = [], []
    exceptions = []
    for row in data:
        labels.append(getLabel(row[2]))
        embedding, exception = getTweetEmbedding(row[1])
        exceptions.append(exception)
        features.append(embedding)
    return labels, features, sum(exceptions)
    
    
dev_labels, dev_features, dev_exceptions = getLabelsFeatures(dev_data)
test_labels, test_features, test_exceptions = getLabelsFeatures(test_data)
train_labels, train_features, train_exceptions = getLabelsFeatures(train_data)


dev_features = np.array(dev_features)
dev_labels = np.array(dev_labels)

test_features = np.array(test_features)
test_labels = np.array(test_labels)

train_features = np.array(train_features)
train_labels = np.array(train_labels)

In [201]:
train_exceptions

5193

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


 def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [209]:
model = Sequential()

# model.add(Dense(512, activation='relu', input_dim=300))
# model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[keras_metrics.precision(), keras_metrics.recall(), f1])

In [246]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# clf = DecisionTreeClassifier(max_depth=10)  # f1 0.54-0.51
# clf = RandomForestClassifier(max_depth=10, n_estimators=10, max_features=5) #f1 0.51-53

# clf = SVC(gamma='auto', random_state=42) # f1 = 0.39
# cross_validate(clf, train_features, train_labels, scoring=['recall_macro', 'precision_macro', 'f1_macro'], cv=2, verbose=True)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/rayanral/PycharmProjects/prj-nlp-2019/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rayanral/PycharmProjects/prj-nlp-2019/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rayanral/PycharmProjects/prj-nlp-2019/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/rayanral/PycharmProjects/prj-nlp-2019/venv/lib/python3.6/site-packages/sklearn/metrics/classificat

{'fit_time': array([10.13283515,  9.60393119]),
 'score_time': array([25.45218301, 25.95975089]),
 'test_recall_macro': array([0.5, 0.5]),
 'train_recall_macro': array([0.5, 0.5]),
 'test_precision_macro': array([0.32671292, 0.32671292]),
 'train_precision_macro': array([0.32671292, 0.32671292]),
 'test_f1_macro': array([0.39519513, 0.39519513]),
 'train_f1_macro': array([0.39519513, 0.39519513])}

In [248]:
clf = DecisionTreeClassifier(max_depth=10)  # f1 0.54-0.51
clf.fit(train_features, train_labels)
clf.score(test_features, test_labels)

0.7386831275720165

In [262]:
for prediction in predictions:
    if prediction[1] >= 0.5:
        print('true\t' + "{0:.4f}".format(prediction[1]))
    else:
        print('false\t' + "{0:.4f}".format(prediction[1]))

true	0.0000
false	0.8726
true	0.2815
false	0.9200
false	0.8384
false	0.8726
false	0.6131
false	1.0000
false	0.5732
true	0.0000
true	0.0000
false	0.8726
false	0.8801
false	0.7253
false	0.5217
false	0.7253
true	0.0000
false	0.7600
false	0.8726
true	0.0000
false	0.7600
false	0.5217
false	0.8726
true	0.1176
false	0.6131
false	0.5217
false	0.7600
true	0.2815
false	0.8726
false	0.9750
false	0.8933
false	0.8532
false	0.8801
false	0.8801
false	0.8801
false	0.8801
false	0.8801
false	0.9222
true	0.0833
true	0.2727
false	0.7600
false	0.9089
false	0.8801
true	0.1008
false	0.8801
true	0.0714
false	0.8801
false	0.9706
false	0.8801
false	0.8726
false	0.8801
false	0.9612
false	0.8801
false	0.8801
false	0.8801
true	0.0000
false	0.8549
false	0.8801
false	0.7992
false	0.6897
false	1.0000
false	0.6600
false	0.8801
true	0.0000
false	0.9200
true	0.2815
true	0.3913
true	0.0000
false	0.7992
true	0.2815
false	0.9200
false	0.8726
false	1.0000
true	0.2815
false	0.7253
false	1.0000
false	0.9853
false	0.9222
false

In [210]:
loss_and_metrics = model.fit(train_features, train_labels, epochs=10, batch_size=10)

print(model.metrics_names)
print(loss_and_metrics)

Epoch 1/10
11530/11530 [==============================] - 4s 374us/step - loss: 0.5749 - precision: 0.6818 - recall: 0.3083 - f1: 0.3381
Epoch 2/10
11530/11530 [==============================] - 4s 333us/step - loss: 0.5210 - precision: 0.7130 - recall: 0.5035 - f1: 0.5362
Epoch 3/10
11530/11530 [==============================] - 4s 349us/step - loss: 0.4965 - precision: 0.7342 - recall: 0.5405 - f1: 0.5720
Epoch 4/10
11530/11530 [==============================] - 4s 343us/step - loss: 0.4752 - precision: 0.7591 - recall: 0.5741 - f1: 0.6067
Epoch 5/10
11530/11530 [==============================] - 4s 367us/step - loss: 0.4618 - precision: 0.7709 - recall: 0.5868 - f1: 0.6185
Epoch 6/10
11530/11530 [==============================] - 4s 347us/step - loss: 0.4487 - precision: 0.7693 - recall: 0.6034 - f1: 0.6223
Epoch 7/10
11530/11530 [==============================] - 5s 423us/step - loss: 0.4366 - precision: 0.7845 - recall: 0.6159 - f1: 0.6375
Epoch 8/10
11530/11530 [=================

In [211]:
loss_and_metrics = model.evaluate(test_features, test_labels, batch_size=10)

print(model.metrics_names)
print(loss_and_metrics)

972/972 [==============================] - 0s 398us/step
['loss', 'precision', 'recall', 'f1']
[0.5109107618300636, 0.17948717902695593, 0.03999999997714286, 0.022854201587629908]


In [212]:
predictions = model.predict(test_features)

In [213]:
for prediction in predictions:
    if prediction >= 0.5:
        print('true\t' + "{0:.4f}".format(prediction[0]))
    else:
        print('false\t' + "{0:.4f}".format(prediction[0]))

false	0.1574
false	0.3024
false	0.1615
false	0.1516
false	0.2027
false	0.3213
false	0.1825
false	0.0469
false	0.2045
false	0.3115
false	0.1228
false	0.1127
false	0.1162
false	0.2243
false	0.0766
false	0.2015
false	0.2397
false	0.2438
false	0.0998
false	0.1173
false	0.1916
false	0.2315
false	0.0643
false	0.3415
false	0.2322
false	0.0791
false	0.1542
false	0.3199
false	0.3305
false	0.1047
false	0.2745
false	0.2954
false	0.0834
false	0.1020
false	0.2898
false	0.3106
false	0.3181
false	0.3314
false	0.1543
false	0.3326
false	0.2629
false	0.2081
false	0.1765
true	0.7000
false	0.2185
false	0.2306
false	0.0941
false	0.2290
false	0.2318
false	0.2938
false	0.3181
false	0.1519
false	0.3802
false	0.1064
false	0.1866
false	0.4654
false	0.3339
false	0.1793
false	0.3029
false	0.3328
false	0.0371
false	0.0344
false	0.0311
false	0.0865
false	0.0044
false	0.0783
false	0.0449
false	0.2245
false	0.1162
false	0.1146
false	0.0013
false	0.0402
false	0.0443
false	0.0783
false	0.0922
false	0.1098
false	0.0150
